In [1]:
# Import required libraries and dependencies
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas

import numpy as np
from scipy.stats import linregress

import matplotlib.pyplot as plt
import hvplot.pandas

In [2]:
### import and read csv file

path = '../../Output/full_indicators_by_country.csv'
full_indicators_countries_df = pd.read_csv(path)
# full_indicators = full_indicators.rename(columns={'total':'Immigration_flow'})

full_indicators_countries_df.head(5)

,index,country,continent,sub-region,clean fuels for cooking (% of population),Access to electricity (% of population),Adolescents out of school (% of lower secondary school age),Agricultural irrigated land (% of total agricultural land),Agricultural land (% ),Arable land (% of land area),...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)","Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,free,partly free,not free,OCDE,immigration_flow,immigration_100k
0,0,Afghanistan,Asia,Southern Asia,32.557143,95.314286,0.000000,6.102359,58.432280,11.936412,...,0.0146,11.936625,20.0,3.294,0,0,1,0,84360,220.482946
1,1,Albania,Europe,Southern Europe,81.700000,99.971429,2.824656,15.210061,42.448107,22.164995,...,4.9900,12.026625,37.0,1.809,0,1,0,0,5740,202.797286
2,2,Algeria,Africa,North Africa,99.685714,99.642857,2.757020,3.184234,17.347773,3.148844,...,0.5940,12.291500,36.0,2.110,0,0,1,0,41860,96.013469
3,3,American Samoa,Oceania,Oceania,0.000000,0.000000,0.000000,0.000000,14.178571,4.742857,...,0.0000,0.000000,0.0,0.000,0,0,0,0,0,0.000000
4,4,Andorra,Europe,Southern Europe,100.000000,100.000000,7.136009,0.000000,39.988997,1.677720,...,10.8140,0.000000,0.0,0.000,1,0,0,0,0,0.000000


In [3]:
# this is to test the list that will be used in the fnction below

indicators_list = full_indicators_countries_df.columns.to_list()
indicators_list = indicators_list[2:len(indicators_list)-1]
indicators_list

['continent',
 'sub-region',
 'clean fuels for cooking (% of population)',
 'Access to electricity (% of population)',
 'Adolescents out of school (% of lower secondary school age)',
 'Agricultural irrigated land (% of total agricultural land)',
 'Agricultural land (% )',
 'Arable land (% of land area)',
 'Arable land (hectares per person)',
 'Armed forces (% labor force)',
 'Armed forces personnel, total',
 'precipitation (mm per year)',
 'Birth rate, crude (per 1,000 people)',
 'Central government debt, total (% of GDP)',
 'Children in employment, total (% of children ages 7-14)',
 'Children out of school (% of primary school age)',
 'Consumer price index (2010 = 100)',
 'Control of Corruption: Estimate',
 'Cost of business start-up (% of GNI per capita)',
 'Death rate, crude (per 1,000 people)',
 'doing business score',
 "Educational attainment, at least Master's or equivalent, population 25+, total (%) (cumulative)",
 'Educational attainment, at least completed primary, population 

In [4]:
# function that compares all the indicators against ONE (immigration flow or immigration_100K) and generate a DF with al the regression parameters

# arguments are 
        ## the dataframe name
        ## number of index columns to be exculded

def select_indicator(dataframe,col_index):
    
    # create list of indicator from the Dataframe consolidated
    indicators_list = dataframe.columns.to_list()
    indicators_list = indicators_list[col_index:len(indicators_list)-1]
    indicators_list

    # create list of returned values
    indic_selection = []

    # loop through the indicators
    for ind in indicators_list :
        col1 = 'immigration_100k'
        col2 = ind


        df_regression = dataframe[[col1,col2]]
        
        #drop empty values

        df_regression = df_regression.dropna(how= 'any')


        # identify outliers
        var = df_regression[col2]
        q1 = np.quantile(var, 0.25) 
        q3 = np.quantile(var, 0.75)   
        iqr = q3 - q1
        outliers = var[(var > (q3 + (iqr * 1.5))) | (var < (q1 - (iqr * 1.5)))].tolist()

        # Exclude  outliers
        x = ~df_regression[col2].isin(outliers)
        dataframe_filtered = df_regression[x]

        x_axis = dataframe_filtered[col1]
        y_axis = dataframe_filtered[col2]

        ## calculate slope and intercept of the regression equation

        (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
        

        # append regresson values in list of dictionaries
        indic_selection.append({
                            'indicator A' : col1,
                            'indicator B' : col2,
                            'slope' : slope,
                            'intercept' :  intercept, 
                            'rvalue' : rvalue, 
                            'pvalue' : pvalue, 
                            'std' : stderr,
                            "rvalue_abs" : abs(rvalue)

                            })
    
    # create DF from list of indicators compared
    selection_df = pd.DataFrame(indic_selection)
    selection_df = selection_df.sort_values('rvalue_abs', ascending=False)

    selection_df = selection_df.reset_index()    
    
    return selection_df.head(50)


# Perform linear regression on full data

In [6]:
# run the function linear regression and compare all indicators by year

Linear_regression_fulldata = select_indicator(full_indicators_countries_df,4)
# synthesis_df.sort_values('pvalue', ascending=False)
Linear_regression_fulldata

,index,indicator A,indicator B,slope,intercept,rvalue,pvalue,std,rvalue_abs
0,36,immigration_100k,Military expenditure (% of GDP),-6.200454e-04,1.262957e+00,-0.194339,0.006622,2.258665e-04,0.194339
1,37,immigration_100k,Population density,5.532327e-02,1.058147e+02,0.169329,0.021212,2.380308e-02,0.169329
2,38,immigration_100k,Population growth (annual %),-6.065820e-04,1.209472e+00,-0.164853,0.019052,2.566230e-04,0.164853
3,42,immigration_100k,Rural population (% of total population),1.007937e-02,3.757933e+01,0.136972,0.050755,5.128774e-03,0.136972
4,48,immigration_100k,free,2.094904e-04,3.747738e-01,0.135695,0.052970,1.076188e-04,0.135695
5,40,immigration_100k,Population in urban agglomerations of more tha...,-5.937219e-03,1.382612e+01,-0.130902,0.066729,3.220088e-03,0.130902
6,41,immigration_100k,"Population, total",-6.137647e+03,1.330106e+07,-0.129855,0.079768,3.483464e+03,0.129855
7,47,immigration_100k,Global_Peace_Index_2023,2.996572e-04,2.124848e+00,0.125990,0.110143,1.865317e-04,0.125990
8,7,immigration_100k,Armed forces (% labor force),-2.828408e-04,7.306480e-01,-0.124058,0.090715,1.663267e-04,0.124058
9,19,immigration_100k,"Educational attainment, at least Master's or e...",-1.760672e-03,3.630247e+00,-0.120002,0.096451,1.053956e-03,0.120002


# Perform regression on Annual indicators data

In [6]:
### import and read csv file

path = '../../Output/full_indicators_by_country_by_year.csv'
full_indicators_countries_year_df = pd.read_csv(path)
# full_indicators = full_indicators.rename(columns={'total':'Immigration_flow'})

full_indicators_countries_year_df.head(5)

,index,country,iso3Code,year,clean fuels for cooking (% of population),Access to electricity (% of population),Agricultural land (% ),Armed forces (% labor force),precipitation (mm per year),"Birth rate, crude (per 1,000 people)",...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)",Total debt service (% of GNI),"Total debt service (% of exports of goods, services and primary income)",Unemployment advanced education,Unemployment basic education,Unemployment intermediate education,"Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,immigration_100k
0,0,Afghanistan,AFG,2015,27.6,71.5,58.123668,3.724667,327.0,38.803,...,0.009,0.292137,3.299608,0.00,0.000,0.000,9.052,20.0,3.294,7.758981
1,1,Afghanistan,AFG,2016,28.8,97.7,58.123668,3.635915,327.0,37.936,...,0.013,0.324903,4.333403,0.00,0.000,0.000,10.133,20.0,3.294,7.651162
2,2,Afghanistan,AFG,2017,30.3,97.7,58.123668,3.501771,327.0,37.342,...,0.017,0.333475,4.353572,15.46,12.283,16.155,11.184,20.0,3.294,9.694882
3,3,Afghanistan,AFG,2018,31.4,93.4,58.276988,2.961201,327.0,36.927,...,0.016,0.344999,3.323800,15.46,12.283,16.155,11.196,20.0,3.294,9.688910
4,4,Afghanistan,AFG,2019,32.6,97.7,58.276988,3.023536,327.0,36.466,...,0.016,0.291267,2.974238,15.46,12.283,16.155,11.185,20.0,3.294,10.275749


In [7]:
# run the function linear regression and compare all indicators by year

Linear_regression_fulldata_year = select_indicator(full_indicators_countries_year_df,3)
# synthesis_df.sort_values('pvalue', ascending=False)
Linear_regression_fulldata_year

,index,indicator A,indicator B,slope,intercept,rvalue,pvalue,std,rvalue_abs
0,22,immigration_100k,government consumption exp (% of GDP),-0.059356,1.492849e+01,-0.282028,3.079109e-31,0.005000,0.282028
1,25,immigration_100k,"Industry (including construction), value added...",-0.064049,2.478084e+01,-0.221990,2.219404e-19,0.007024,0.221990
2,32,immigration_100k,% of total population below 5 meters,0.022581,3.006315e+00,0.217722,9.345048e-18,0.002599,0.217722
3,29,immigration_100k,Military expenditure (% of GDP),-0.006048,1.320856e+00,-0.213247,1.237412e-17,0.000699,0.213247
4,30,immigration_100k,Population density,0.571375,1.007920e+02,0.196144,1.301345e-14,0.073412,0.196144
5,31,immigration_100k,Population growth (annual %),-0.006220,1.263138e+00,-0.168658,6.233488e-12,0.000898,0.168658
6,34,immigration_100k,"Population, total",-71458.776796,1.505780e+07,-0.156173,1.322853e-09,11708.317463,0.156173
7,4,immigration_100k,Armed forces (% labor force),-0.003012,7.986952e-01,-0.151173,3.394282e-09,0.000507,0.151173
8,33,immigration_100k,Population living in slums (% of urban populat...,-0.065953,1.231282e+01,-0.128734,3.279021e-07,0.012859,0.128734
9,35,immigration_100k,Poverty $2.15 a day (% of population),-0.002878,5.328798e-01,-0.109116,5.735474e-05,0.000713,0.109116


# Perform Linear regression on clustered data

In [7]:
### import and read csv file

path = '../../Output_analysis/Clusters_analysis.csv'
clusters = pd.read_csv(path)
# full_indicators = full_indicators.rename(columns={'total':'Immigration_flow'})

clusters.head(5)

,country,countries_cluster_original_data,countries_cluster_withPCA
0,Afghanistan,0,0
1,Lesotho,0,0
2,Zambia,0,0
3,Kenya,0,0
4,Honduras,0,0


In [8]:
new_df = pd.merge(full_indicators_countries_df, clusters, on = 'country' )
new_df

,index,country,continent,sub-region,clean fuels for cooking (% of population),Access to electricity (% of population),Adolescents out of school (% of lower secondary school age),Agricultural irrigated land (% of total agricultural land),Agricultural land (% ),Arable land (% of land area),...,Corruption index 2023,Global_Peace_Index_2023,free,partly free,not free,OCDE,immigration_flow,immigration_100k,countries_cluster_original_data,countries_cluster_withPCA
0,0,Afghanistan,Asia,Southern Asia,32.557143,95.314286,0.000000,6.102359,58.432280,11.936412,...,20.0,3.294,0,0,1,0,84360,220.482946,0,0
1,1,Albania,Europe,Southern Europe,81.700000,99.971429,2.824656,15.210061,42.448107,22.164995,...,37.0,1.809,0,1,0,0,5740,202.797286,1,3
2,2,Algeria,Africa,North Africa,99.685714,99.642857,2.757020,3.184234,17.347773,3.148844,...,36.0,2.110,0,0,1,0,41860,96.013469,2,1
3,3,American Samoa,Oceania,Oceania,0.000000,0.000000,0.000000,0.000000,14.178571,4.742857,...,0.0,0.000,0,0,0,0,0,0.000000,2,2
4,4,Andorra,Europe,Southern Europe,100.000000,100.000000,7.136009,0.000000,39.988997,1.677720,...,0.0,0.000,1,0,0,0,0,0.000000,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,199,Viet Nam,Asia,South-Eastern Asia,92.671429,99.771429,3.785817,0.000000,39.155435,21.892959,...,41.0,1.802,0,0,1,0,43215,44.343809,2,1
206,200,West Bank and Gaza,Asia,Western Asia,0.000000,99.985714,5.636399,0.000000,69.685212,8.563669,...,0.0,2.872,0,0,1,0,6090,128.171093,2,1
207,201,"Yemen, Rep.",Asia,Western Asia,52.514286,72.528571,34.258968,0.000000,44.401278,2.180265,...,16.0,3.397,0,0,1,0,6145,17.231873,0,0
208,202,Zambia,Africa,Eastern Africa,11.142857,42.571429,0.000000,0.000000,32.048742,5.092501,...,37.0,1.948,0,1,0,0,630,3.351441,0,0


In [9]:
# to have the population of each cluster
x = new_df.groupby('countries_cluster_withPCA')['country'].count()
x

countries_cluster_withPCA
0    50
1    61
2    40
3    59
Name: country, dtype: int64

In [11]:
# create empty list to store the results
results_list = []

# loop for each cluster and generate a linear regression DF
for i in range(0,4) :
    
    result_dict = {}
    
    cluster_num = i

    # filter the DF regarding the cluster

    clustered_df = new_df[new_df['countries_cluster_withPCA']==cluster_num]
    
    # prepare the DF to the linear regression
    clustered_df = clustered_df.drop(columns=['countries_cluster_original_data', 'countries_cluster_withPCA'])
    
    # run the linear regression
    Linear_regression_with_PCA = select_indicator(clustered_df,4)

    # export the reslt of the linear regression to EXCEL file
    # file_destination = f'../../Output_analysis/Linear_regression_with_PCA_cluster_number_{cluster_num}.xlsx'
    # Linear_regression_with_PCA.to_excel(file_destination)

    # extract the max rvalue
    rvalue_max = Linear_regression_with_PCA['rvalue_abs'].max()

    # store the max rvalue in a dict to be appended in the list results
    result_dict = {
            'cluster' : i ,
            'rvalue' : rvalue_max
                }
    results_list.append(result_dict)

# create DF with list of results
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('rvalue', ascending=False)
results_df

,cluster,rvalue
2,2,0.849723
3,3,0.395994
1,1,0.332537
0,0,0.308141


In [23]:
# filter the DF regarding the cluster

## choose the cluster
cluster_num = 1

## filter
clustered_df = new_df[new_df['countries_cluster_withPCA']==cluster_num]
clustered_df = clustered_df.drop(columns=['countries_cluster_original_data', 'countries_cluster_withPCA'])
clustered_df

,index,country,continent,sub-region,clean fuels for cooking (% of population),Access to electricity (% of population),Adolescents out of school (% of lower secondary school age),Agricultural irrigated land (% of total agricultural land),Agricultural land (% ),Arable land (% of land area),...,"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)","Unemployment, total (% of total labor force) (modeled ILO estimate)",Corruption index 2023,Global_Peace_Index_2023,free,partly free,not free,OCDE,immigration_flow,immigration_100k
2,2,Algeria,Africa,North Africa,99.685714,99.642857,2.757020,3.184234,17.347773,3.148844,...,0.5940,12.291500,36.0,2.110,0,0,1,0,41860,96.013469
11,11,Azerbaijan,Asia,Western Asia,97.971429,100.000000,7.603029,30.187448,57.814917,25.152036,...,1.8960,5.558250,23.0,2.248,0,0,1,0,2560,25.567247
13,13,Bahrain,Asia,Western Asia,100.000000,100.000000,2.431848,0.000000,10.367165,2.677938,...,1.5660,1.326375,42.0,2.072,0,0,1,0,130,8.673888
14,14,Bangladesh,Asia,Southern Asia,23.542857,91.085714,13.146852,68.375204,71.606185,60.203514,...,0.0224,4.672500,24.0,2.126,0,1,0,0,35490,21.407842
22,22,Bolivia,South America,South America,86.442857,95.314286,12.122364,0.788924,35.437552,4.743888,...,4.0480,4.240250,29.0,2.009,0,1,0,0,1200,10.236797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,186,Turkiye,Europe,Southern Europe,95.128571,100.000000,0.278877,13.744554,49.400918,25.864748,...,1.8020,11.434250,34.0,2.780,0,0,1,1,21945,26.543086
198,192,United Arab Emirates,Asia,Western Asia,100.000000,100.000000,17.668720,22.798855,5.456558,0.665406,...,2.3360,2.636375,68.0,1.897,0,0,1,0,50,0.522973
202,196,Uzbekistan,Asia,Central Asia,80.828571,99.971429,3.569006,14.542578,58.132461,9.137510,...,2.5060,5.114750,33.0,1.957,0,0,1,0,1610,4.826442
205,199,Viet Nam,Asia,South-Eastern Asia,92.671429,99.771429,3.785817,0.000000,39.155435,21.892959,...,9.0400,1.777500,41.0,1.802,0,0,1,0,43215,44.343809


In [24]:
# run the function linear regression and compare all indicators by year

Linear_regression_with_PCA = select_indicator(clustered_df,4)
# synthesis_df.sort_values('pvalue', ascending=False)
Linear_regression_with_PCA

,index,indicator A,indicator B,slope,intercept,rvalue,pvalue,std,rvalue_abs
0,47,immigration_100k,Global_Peace_Index_2023,8.195940e-04,2.043276e+00,0.332537,0.013118,3.192816e-04,0.332537
1,27,immigration_100k,GDP growth (annual %),-1.048215e-02,3.529346e+00,-0.326993,0.011476,4.012540e-03,0.326993
2,50,immigration_100k,not free,-1.487786e-03,6.154136e-01,-0.302649,0.017752,6.099797e-04,0.302649
3,2,immigration_100k,Adolescents out of school (% of lower secondar...,1.982357e-02,5.353219e+00,0.261210,0.047643,9.789313e-03,0.261210
4,1,immigration_100k,Access to electricity (% of population),-5.525857e-03,9.897489e+01,-0.235136,0.086981,3.167590e-03,0.235136
5,38,immigration_100k,Population growth (annual %),-2.017954e-03,1.172810e+00,-0.231612,0.072489,1.103445e-03,0.231612
6,18,immigration_100k,doing business score,-1.784052e-02,6.447051e+01,-0.222392,0.093367,1.045154e-02,0.222392
7,7,immigration_100k,Armed forces (% labor force),1.805412e-03,1.061388e+00,0.218042,0.103232,1.089625e-03,0.218042
8,45,immigration_100k,"Unemployment, total (% of total labor force) (...",1.146949e-02,6.674651e+00,0.217152,0.098521,6.828944e-03,0.217152
9,9,immigration_100k,precipitation (mm per year),1.722692e+00,8.398545e+02,0.215649,0.095091,1.015531e+00,0.215649
